## **1. Find the corresponding positive values for NER, POS, Chunk tags**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **2. Data Preprocessing for BERT Model (Apply Hugging Face Data)**

### (1) Hugging Face Dataset Conll2003 Exploration

In [ ]:
!pip install datasets

     |████████████████████████████████| 298 kB 5.1 MB/s 
     |████████████████████████████████| 132 kB 67.4 MB/s 
     |████████████████████████████████| 1.1 MB 66.3 MB/s 
     |████████████████████████████████| 61 kB 654 kB/s 
     |████████████████████████████████| 243 kB 79.1 MB/s 
     |████████████████████████████████| 271 kB 77.5 MB/s 
     |████████████████████████████████| 192 kB 78.0 MB/s 
     |████████████████████████████████| 160 kB 79.9 MB/s 


In [ ]:
from datasets import load_dataset
# dataset = load_dataset('conll2003')

### (2) Covert Data to BERT Input Style

In [ ]:
!pip install transformers seqeval[gpu]

     |████████████████████████████████| 3.3 MB 5.0 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 596 kB 74.2 MB/s 
     |████████████████████████████████| 895 kB 75.5 MB/s 
     |████████████████████████████████| 3.3 MB 71.7 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=c70b26576999fea0c696f037a028de4b98726647c3f76aec4f3442813f14abf3
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

In [ ]:
MAX_LEN = 128     
TRAIN_BATCH_SIZE = 4
TEST_BATCH_SIZE = 2
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/NLP/Final Project/New Dataset/Bank/ner_datasetreference_new.csv", encoding='unicode_escape')
data.head()

data.count()

Sentence #      47959
Word          1048575
POS           1048575
Tag           1048575
dtype: int64

In [ ]:
'''
step 2a: process NE tags and POS tags
'''
# NE 
"""There are 8 category tags, each with a "beginning" and "inside" variant, and the "outside" tag. It is not really clear what these tags mean - "geo" probably stands for geographical entity, "gpe" for geopolitical entity, and so on. They do not seem to correspond with what the publisher says on Kaggle. Some tags seem to be underrepresented. Let's print them by frequency (highest to lowest): """

# tags = {}
# for tag, count in zip(frequencies_NE.index, frequencies_NE):
#     if tag != "O":
#         if tag[2:5] not in tags.keys():
#             tags[tag[2:5]] = count
#         else:
#             tags[tag[2:5]] += count
#     continue

# print(sorted(tags.items(), key=lambda x: x[1], reverse=True))

"""Let's remove "art", "eve" and "nat" named entities, as performance on them will probably be not comparable to the other named entities. """

entities_to_remove = ["B-art", "I-art", "B-eve", "I-eve", "B-nat", "I-nat"]
data = data[~data.Tag.isin(entities_to_remove)]
data.head()
data.count()


Sentence #      47920
Word          1047063
POS           1047063
Tag           1047063
dtype: int64

In [ ]:
"""We create 2 dictionaries for NE: one that maps individual tags to indices, and one that maps indices to their individual tags. This is necessary in order to create the labels (as computers work with numbers = indices, rather than words = tags) - see further in this notebook."""

labels_to_ids = {k: v for v, k in enumerate(data.Tag.unique())}
ids_to_labels = {v: k for v, k in enumerate(data.Tag.unique())}
print(labels_to_ids)
print(ids_to_labels)


{'O': 0, 'B-geo': 1, 'B-gpe': 2, 'B-per': 3, 'I-geo': 4, 'B-org': 5, 'I-org': 6, 'B-tim': 7, 'I-per': 8, 'I-gpe': 9, 'I-tim': 10}
{0: 'O', 1: 'B-geo', 2: 'B-gpe', 3: 'B-per', 4: 'I-geo', 5: 'B-org', 6: 'I-org', 7: 'B-tim', 8: 'I-per', 9: 'I-gpe', 10: 'I-tim'}


In [ ]:
# count NE tag
print("Number of NE tags: {}".format(len(data.Tag.unique()))) # 17个
frequencies_NE = data.Tag.value_counts()
frequencies_NE
Ner_Tag = list(data.Tag.unique())
Ner_Number = [i for i in range(len(Ner_Tag))]
Ner = list(zip(Ner_Tag,Ner_Number))
print(Ner)


Number of NE tags: 11
[('O', 0), ('B-geo', 1), ('B-gpe', 2), ('B-per', 3), ('I-geo', 4), ('B-org', 5), ('I-org', 6), ('B-tim', 7), ('I-per', 8), ('I-gpe', 9), ('I-tim', 10)]


In [ ]:
# pandas has a very handy "forward fill" function to fill missing values based on the last upper non-nan value
data = data.fillna(method='ffill')
data.head()


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [ ]:
# let's create a new column called "sentence" which groups the words by sentence 
data['sentence'] = data[['Sentence #','Word','Tag', 'POS']].groupby(['Sentence #'])['Word'].transform(lambda x: ' '.join(x))
# let's also create a new column called "word_labels" which groups the tags by sentence 
data['word_labels'] = data[['Sentence #','Word','Tag', 'POS']].groupby(['Sentence #'])['Tag'].transform(lambda x: ','.join(x))
data.head()


,Sentence #,Word,POS,Tag,sentence,word_labels
0,Sentence: 1,Thousands,NNS,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
1,Sentence: 1,of,IN,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
2,Sentence: 1,demonstrators,NNS,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
3,Sentence: 1,have,VBP,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
4,Sentence: 1,marched,VBN,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."


In [ ]:
"""Let's only keep the "sentence" and "word_labels" columns, and drop duplicates:"""

data = data[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
data.head()

len(data)
"""Let's verify that a random sentence and its corresponding tags are correct:"""

print(data.iloc[41].sentence)
print(data.iloc[41].word_labels)


Bedfordshire police said Tuesday that Omar Khayam was arrested in Bedford for breaching the conditions of his parole .
B-gpe,O,O,B-tim,O,B-per,I-per,O,O,O,B-geo,O,O,O,O,O,O,O,O


In [ ]:
train_df, validate_df, test_df = \
              np.split(data.sample(frac=1, random_state=42), 
                       [int(.85*len(data)), int(.925*len(data))])


In [ ]:
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

data_combine_dict = {'train':train_df, 'validation':validate_df, 'test':test_df}


In [ ]:
class Preprocess_Data(Dataset):
  def __init__(self, dataset, tokenizer, max_len): #usage -> train, validation, test

        self.len = len(dataset)
        self.data = dataset
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):

        # step 1: get the sentence and word labels 
        sentence = self.data.sentence[index].strip().split()
        word_labels = self.data.word_labels[index].split(",")


        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                              is_split_into_words=True,
                              return_offsets_mapping=True,  #Set to True to return (char_start, char_end) for each token (default False)
                              padding='max_length', 
                              truncation=True, 
                              max_length=self.max_len)
        
        
        # step 3: create token labels only for first word pieces of each tokenized word

        labels = [labels_to_ids[label] for label in word_labels]

        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
        
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        
        return item

  def __len__(self):
        return self.len

In [ ]:
dataset = data_combine_dict

training_set = Preprocess_Data(train_df, tokenizer, MAX_LEN)
validation_set = Preprocess_Data(validate_df, tokenizer, MAX_LEN)
testing_set = Preprocess_Data(test_df, tokenizer, MAX_LEN)
print(len(training_set),len(validation_set),len(testing_set))


40435 3568 3568


In [ ]:
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["input_ids"]), training_set[0]["labels"]):
  print('{0:10}  {1}'.format(token, label))

[CLS]       -100
The         0
Tibetan     2
spiritual   0
leader      0
who         0
teaches     0
age         0
-           -100
old         -100
principles  0
of          0
peace       0
and         0
tolerance   0
has         0
gone        0
high        0
-           -100
tech        -100
,           0
joining     0
the         0
online      0
mess        0
##aging     -100
service     0
.           0
[SEP]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       

In [ ]:
# Define the Dataloader
training_loader = DataLoader(training_set, batch_size = TRAIN_BATCH_SIZE, shuffle=True,num_workers=0)
validation_loader = DataLoader(validation_set,batch_size = TRAIN_BATCH_SIZE, shuffle=True,num_workers=0)
testing_loader = DataLoader(testing_set,batch_size = TEST_BATCH_SIZE, shuffle=True,num_workers=0)

print(len(training_loader),len(validation_loader),len(testing_loader))


10109 892 1784


In [ ]:
# Define the Dataloader
training_loader = DataLoader(training_set, batch_size = TRAIN_BATCH_SIZE, shuffle=True,num_workers=0)
validation_loader = DataLoader(validation_set,batch_size = TRAIN_BATCH_SIZE, shuffle=True,num_workers=0)
testing_loader = DataLoader(testing_set,batch_size = TEST_BATCH_SIZE, shuffle=True,num_workers=0)

In [ ]:
print(len(training_loader),len(validation_loader),len(testing_loader))

10109 892 1784


# **3. Define the Model**

### 1) Train the Model

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Dec 11 00:46:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    42W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
# EPOCHS = 2
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10

In [ ]:
# Define the model by just BertForTokenClassification
model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(Ner_Tag))
model.to(device)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [ ]:
import time 
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):

        # if idx >200:
        #   break
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs[0]
        tr_logits = outputs[1]
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
            if idx != 0:
              time_spent = time.time() - start_time
              print("--- %s seconds ---" % (time_spent))
            start_time = time.time()

           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

        #print('flattened_targets', flattened_targets)
        #print('active_logits, ', active_logits)
        #print('flattened_predictions, ', flattened_predictions)
        # print('Logits 0, ', active_logits[0])
        # # print("real labels",flattened_targets)
        # # print("real prediction",flattened_predictions)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        #print('active_accuracy ', active_accuracy)

        labels = torch.masked_select(flattened_targets, active_accuracy)
        #print('labels ', labels)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        #print('predictions ',predictions)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

    # --------------------------------------------------------------------------------------------------------------------
    # After the completion of each training epoch
    # measure our performance on validation set.

    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(validation_loader):

            # if idx >200:
            #   break
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)

            
            outputs= model(input_ids=ids, attention_mask=mask, labels=labels)
            loss = outputs[0]
            eval_logits = outputs[1]
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

In [ ]:
len(training_loader),len(validation_loader),len(testing_loader)

(10109, 892, 1784)

In [ ]:
from seqeval.metrics import classification_report
New_NerDict = dict((v,k) for k,v in dict(Ner).items())
New_NerDict

{0: 'O',
 1: 'B-geo',
 2: 'B-gpe',
 3: 'B-per',
 4: 'I-geo',
 5: 'B-org',
 6: 'I-org',
 7: 'B-tim',
 8: 'I-per',
 9: 'I-gpe',
 10: 'I-tim'}

In [ ]:
import os
import time

In [ ]:
EPOCHS = 3
for epoch in range(EPOCHS):

    directory = "/content/drive/MyDrive/NLP/Final Project/New Dataset/Bank/Baseline/Cased/Model_"+str(epoch+1)

    if not os.path.exists(directory):
        os.makedirs(directory)


    print(f"Training epoch: {epoch + 1}")
    train(epoch)
    labels, predictions = valid(model, testing_loader)

    labels_value = [[New_NerDict[i.item()] for i in labels]]
    pred_value = [[New_NerDict[i.item()] for i in predictions]]

    print(classification_report(labels_value, pred_value))

    tokenizer.save_vocabulary(directory)
    # save the model weights and its configuration file
    model.save_pretrained(directory)
    print('All files saved')

Training epoch: 1
Training loss per 100 training steps: 2.4574403762817383
Training loss per 100 training steps: 0.8305661631102609
--- 6.259000062942505 seconds ---
Training loss per 100 training steps: 0.5829148917871328
--- 6.2004570960998535 seconds ---
Training loss per 100 training steps: 0.46191439074634316
--- 6.196728467941284 seconds ---
Training loss per 100 training steps: 0.39226172004647536
--- 6.3908796310424805 seconds ---
Training loss per 100 training steps: 0.3509406074546947
--- 6.208604097366333 seconds ---
Training loss per 100 training steps: 0.3137690831989223
--- 6.217293739318848 seconds ---
Training loss per 100 training steps: 0.29061334599181476
--- 6.201436519622803 seconds ---
Training loss per 100 training steps: 0.2725380587109019
--- 6.1972815990448 seconds ---
Training loss per 100 training steps: 0.25721586787541884
--- 6.222629547119141 seconds ---
Training loss per 100 training steps: 0.24495543834132763
--- 6.386406898498535 seconds ---
Training l

### 2) Evaluate the Model

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)

        #             outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        # loss = outputs[0]
        # tr_logits = outputs[1]
        # tr_loss += loss.item()
            
            outputs= model(input_ids=ids, attention_mask=mask, labels=labels)
            loss = outputs[0]
            eval_logits = outputs[1]

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
                if idx!=0:
                  print("--- %s seconds ---" % (time.time() - start_time))
                start_time = time.time()
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy


    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Testing Loss: {eval_loss}")
    print(f"Testing Accuracy: {eval_accuracy}")

    return eval_labels, eval_preds


In [ ]:
len(testing_loader)

1727

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/pytorch_model.bin'))

<All keys matched successfully>

In [ ]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 3.179767608642578
Validation loss per 100 evaluation steps: 0.1595110519685999
Validation loss per 100 evaluation steps: 0.12362539394546082
Validation loss per 100 evaluation steps: 0.11389398336419418
Validation loss per 100 evaluation steps: 0.11897770675265595
Validation loss per 100 evaluation steps: 0.11645176817837057
Validation loss per 100 evaluation steps: 0.11470924197494117
Validation loss per 100 evaluation steps: 0.1201461836735611
Validation loss per 100 evaluation steps: 0.11642353929331893


KeyboardInterrupt: ignored

In [ ]:
New_NerDict = dict((v,k) for k,v in dict(Ner).items())
New_NerDict

In [ ]:
labels_value = [[New_NerDict[i.item()] for i in labels]]
pred_value = [[New_NerDict[i.item()] for i in predictions]]
print(labels_value)
print(pred_value)

In [ ]:
from seqeval.metrics import classification_report

print(classification_report(labels_value, pred_value))

In [ ]:
pip install seqeval

     |████████████████████████████████| 43 kB 2.0 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=104e7548622562892f5d00dea1c4bb4e75e5274c8dc1b4f317572613f23894b0
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
from datasets import load_metric
metric = load_metric("seqeval")

In [ ]:
metric

Metric(name: "seqeval", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of shape (n_samples,), weights for indi

### 3) Save Model

In [ ]:
import os

directory = "./model"

if not os.path.exists(directory):
    os.makedirs(directory)

# save vocabulary of the tokenizer
tokenizer.save_vocabulary(directory)
# save the model weights and its configuration file
model.save_pretrained(directory)
print('All files saved')

In [ ]:
#torch.save(model, 'model.pth')

#torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
